In [3]:
import sys
!{sys.executable} -m pip install sqlalchemy

     |████████████████████████████████| 1.3 MB 1.9 MB/s eta 0:00:01


In [14]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
table_names = engine.table_names()
print(table_names)
con = engine.connect()

['Bookings', 'Facilities', 'Members']


In [17]:
'''Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!'''
query = '''SELECT f.name AS facility,
  SUM(CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
  ELSE f.membercost * b.slots END) AS total_revenue
FROM Facilities f
INNER JOIN Bookings b ON b.facid = f.facid
GROUP BY facility
ORDER BY total_revenue'''
rs = con.execute(query)
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
df

,facility,total_revenue
0,Table Tennis,180.0
1,Snooker Table,240.0
2,Pool Table,270.0
3,Badminton Court,1906.5
4,Squash Court,13468.0
5,Tennis Court 1,13860.0
6,Tennis Court 2,14310.0
7,Massage Room 2,14454.6
8,Massage Room 1,50351.6


In [18]:
'''Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order'''
query = '''SELECT m.surname || ',' || m.firstname AS member,
    (m2.surname || ',' || m2.firstname) AS recommendedby
FROM Members m
INNER JOIN Members m2 ON m2.memid = m.recommendedby
ORDER BY recommendedby'''
rs = con.execute(query)
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
df

,member,recommendedby
0,"Sarwin,Ramnaresh","Bader,Florence"
1,"Coplin,Joan","Baker,Timothy"
2,"Genting,Matthew","Butters,Gerald"
3,"Baker,Timothy","Farrell,Jemima"
4,"Pinker,David","Farrell,Jemima"
5,"Rumney,Henrietta","Genting,Matthew"
6,"Jones,Douglas","Jones,David"
7,"Dare,Nancy","Joplette,Janice"
8,"Jones,David","Joplette,Janice"
9,"Hunt,John","Purview,Millicent"


In [19]:
'''Q12: Find the facilities with their usage by member, but not guests '''
query = '''SELECT f.name AS facility,
       m.surname || ',' || m.firstname AS member,
       COUNT(*) "usage"
FROM Facilities f
INNER JOIN Bookings b ON f.facid = b.facid
INNER JOIN Members m ON m.memid = b.memid
WHERE b.memid <> 0
GROUP BY facility, member'''
rs = con.execute(query)
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
df

,facility,member,usage
0,Badminton Court,"Bader,Florence",9
1,Badminton Court,"Baker,Anne",10
2,Badminton Court,"Baker,Timothy",7
3,Badminton Court,"Boothe,Tim",12
4,Badminton Court,"Butters,Gerald",20
...,...,...,...
197,Tennis Court 2,"Smith,Darren",19
198,Tennis Court 2,"Smith,Jack",1
199,Tennis Court 2,"Smith,Tracy",2
200,Tennis Court 2,"Stibbons,Ponder",31


In [20]:
'''Q13: Find the facilities usage by month, but not guests'''
query = '''SELECT f.name AS facility,
       m.surname || ',' || m.firstname AS member,
       strftime('%m', b.starttime) AS "month",
       count(*) AS "usage"
FROM Facilities f
INNER JOIN Bookings b ON f.facid = b.facid
INNER JOIN Members m ON m.memid = b.memid
WHERE b.memid <> 0
GROUP BY facility, member, "month"
ORDER BY "month";'''
rs = con.execute(query)
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
df

,facility,member,month,usage
0,Badminton Court,"Boothe,Tim",07,1
1,Badminton Court,"Butters,Gerald",07,4
2,Badminton Court,"Dare,Nancy",07,2
3,Badminton Court,"Rownam,Tim",07,1
4,Badminton Court,"Smith,Darren",07,38
...,...,...,...,...
367,Tennis Court 2,"Rumney,Henrietta",09,1
368,Tennis Court 2,"Sarwin,Ramnaresh",09,11
369,Tennis Court 2,"Smith,Darren",09,3
370,Tennis Court 2,"Smith,Jack",09,1


In [21]:
con.close()